In [81]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import rc
rc('font', family='AppleGothic')
%matplotlib inline
import seaborn as sns
import pandas_profiling as pp
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
#plt.rc('font', family='NanumGothic', size= 20)

data1=pd.read_excel('master_data분석용.xlsx')
data=data1

# PREPROCESSING

In [82]:
# 이상치 제거

# 달성률>5000% 제거
# 펀딩기간일 0 - 2개제거
# 마감배송일 -1넘어가면 로그취했을 때 inf 발생

data=data[(data.펀딩기간일!=0)&(data.펀딩기간일<75)]
data=data[data.달성률<5000]
data=data[data['마감배송차이']>-1]
data=data.reset_index(drop=True)

In [83]:
#히스토그램 정규분포에 가까운 독립변수9(펀딩기간일,Strong,Under,문장수,리워즈종류수,마감배송차이,와디즈기사개수,가독성,이미지수)와 종속변수1(달성률) 로그화
beforescale=['펀딩기간일','Strong','Under','문장수','달성률','리워드종류수','마감배송차이','와디즈기사개수','가독성2','이미지수']
afterscale=['펀딩기간일_T','Strong_T','Under_T','문장수_T','달성률_T','리워드종류수_T','마감배송차이_T','와디즈기사개수_T','가독성2_T','이미지수_T']
for i,j in zip(beforescale,afterscale):
    data[j]=preprocessing.scale(np.log(data[i]+1))
    
data['가독성_T']=RobustScaler().fit_transform(data.가독성.values.reshape(-1, 1))

In [84]:
# 달성률 분류
q1, q2, q3 = np.percentile(data.달성률, [25, 50, 75])
target_list = []

for i in range(len(data['달성률'])):
    if data['달성률'][i] < 100:
        target_list.append(0)
    elif data['달성률'][i] < 300:
        target_list.append(1)
    elif data['달성률'][i] < 900:
        target_list.append(2)
    elif data['달성률'][i] < 5000:
        target_list.append(3)
    else:
        target_list.append(4)
    
data['달성률_C'] = target_list

In [65]:
# 달성률_T 분류
q1, q2, q3 = np.percentile(data.달성률_T, [25, 50, 75])
target_list = []

for i in range(len(data['달성률_T'])):
    if data['달성률_T'][i] < q1:
        target_list.append(0)
    elif data['달성률_T'][i] < q2:
        target_list.append(1)
    elif data['달성률_T'][i] < q3:
        target_list.append(2)
    else:
        target_list.append(3)
   
# for i in range(len(data['달성률'])):
#    if data['달성률'][i] < 100:
#        target_list.append(0)
#    elif data['달성률'][i] < 300:
#        target_list.append(1)
#    elif data['달성률'][i] < 900:
#        target_list.append(2)
#    elif data['달성률'][i] < 5000:
#        target_list.append(3)
#   else:
#        target_list.append(4)
    
data['달성률_C'] = target_list

# ENCODING

In [85]:
# 펀딩시작월 mapping
펀딩시작월_dict = {1:'Jan', 2:'Feb', 3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
data['펀딩시작월'] = data.펀딩시작월.map(펀딩시작월_dict)

In [86]:
categorical_feature_names= ['앵콜펀딩여부','펀딩성공여부']
for col in categorical_feature_names:
    data[col] = data[col].astype("category")

In [87]:
# Multi변수 명목변수화-원핫인코딩
펀딩시작월 = pd.get_dummies(data['펀딩시작월'])
data=pd.concat([data, 펀딩시작월], axis=1)
del data['펀딩시작월']

카테고리 = pd.get_dummies(data['카테고리'])
data=pd.concat([data, 카테고리], axis=1)
del data['카테고리']

펀딩시작요일 = pd.get_dummies(data['펀딩시작요일'])
data=pd.concat([data, 펀딩시작요일], axis=1)
del data['펀딩시작요일']

In [88]:
feature_names=['와디즈기사개수_T',
               #'좋아요수',
               '목표금액', 
               '리워드종류수_T',
               '이미지수_T',
               '비디오수',
               '마감배송차이_T',
               '와디즈팔로워수',
               '과거프로젝트수',
               '과거성공프로젝트수',
               '앵콜펀딩여부',
               #'음절수',
               #'단어수',
               '문장수_T',
               'Strong_T',
               'Under_T',
               '가독성_T',
               '가독성2_T', 
               '제목단어수', 
               '펀딩성공여부',
               '디자인소품',
               '반려동물',
               '뷰티',
               '스포츠·모빌리티',
              '여행·레저',
               '테크·가전',
               '패션·잡화',
               '푸드', 
               '홈리빙',
               'Fri',
               'Mon',
               'Sat',
               'Sun',
               'Thu',
               'Tue',
               'Wed',
               '펀딩기간일_T',
               '일펀딩금액',
               '일글수',
              '문장당강조',
               '문장당밑줄',
              'Jan',
              'Feb',
              'Mar',
              'Apr',
              'May',
              'June',
              'July',
              'Aug',
              'Sep',
              'Oct',
              'Nov',
              'Dec']
data_f = data[feature_names]

# CLASSIFICATION

In [89]:
X = data_f
y = data.달성률_C

# 학습용/테스트용 데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.3, random_state=100)

In [90]:
# 학습 모델 결정 및 학습 - RF Classification
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=300, 
                               max_depth = 5, n_jobs = -1)
rf_clf.fit(X_train, y_train)
rf_pred= rf_clf.predict(X_test)                     
                     
rf_clf.score(X_test, y_test)
                     
import sklearn.metrics as mt
print('accuracy', mt.accuracy_score(y_test,rf_pred) )
#print('precision', mt.precision_score(y_test,rf_pred) )
#print('recall', mt.recall_score(y_test,rf_pred) )
#print('f1', mt.f1_score(y_test,rf_pred) )
print(mt.classification_report(y_test,rf_pred))
print(mt.confusion_matrix(y_test,rf_pred))

accuracy 0.6493421052631579
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       353
           1       0.53      0.87      0.66       447
           2       0.44      0.15      0.22       383
           3       0.61      0.56      0.58       337

   micro avg       0.65      0.65      0.65      1520
   macro avg       0.65      0.64      0.62      1520
weighted avg       0.63      0.65      0.61      1520

[[353   0   0   0]
 [  0 388  30  29]
 [  0 233  57  93]
 [  0 105  43 189]]


In [91]:
# 변수간 중요도 보기
feature=pd.DataFrame(data.columns)
importance=pd.DataFrame(rf_clf.feature_importances_)
중요도 =pd.concat([feature,importance],axis=1)
중요도.columns=['Feature','중요도']
중요도.sort_values(ascending=False, by='중요도')

,Feature,중요도
16,문장수,0.426984
35,시작15일차이,0.217982
6,이미지수,0.086786
3,목표금액,0.043085
1,펀딩마감날짜,0.031205
34,시작10일차이,0.024944
10,와디즈팔로워수,0.023673
8,마감배송차이,0.022162
2,펀딩기간일,0.016912
11,과거프로젝트수,0.013427


In [ ]:
# 변수간 중요도 시각화 함수
def plot_feature_importances_cancer(model):
    n_features = cancer.data.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), cancer.feature_names)
    plt.xlabel("특성 중요도")
    plt.ylabel("특성")
    plt.ylim(-1, n_features)

plot_feature_importances_cancer(tree)

In [ ]:
%matplotlib inline
import seaborn as sns
g = sns.pairplot(X, hue=y)
g
plt.show()

# 변수추가

In [ ]:
addfeatures=['시작1일차이','시작5일차이','시작10일차이','시작15일차이','시작20일차이','시작25일차이','마감1일차이','마감5일차이','마감10일차이',\
             '마감15일차이','마감20일차이','마감25일차이']
addfeatures_scaled=['시작1일차이_T','시작5일차이_T','시작10일차이_T','시작15일차이_T','시작20일차이_T','시작25일차이_T','마감1일차이_T','마감5일차이_T','마감10일차이_T',\
                   '마감15일차이_T','마감20일차이_T','마감25일차이_T']

for i,j in zip(addfeatures,addfeatures_scaled):
    data[j]=preprocessing.scale(np.log(data[i]+1))

In [ ]:
feature_names=['와디즈기사개수_T',
               #'좋아요수',
               '목표금액', 
               '리워드종류수_T',
               '이미지수_T',
               '비디오수',
               '마감배송차이_T',
               '와디즈팔로워수',
               '과거프로젝트수',
               '과거성공프로젝트수',
               '앵콜펀딩여부',
               #'음절수',
               #'단어수',
               '문장수_T',
               'Strong_T',
               'Under_T',
               '가독성_T',
               '가독성2_T', 
               '제목단어수', 
               '펀딩성공여부',
               '디자인소품',
               '반려동물',
               '뷰티',
               '스포츠·모빌리티',
              '여행·레저',
               '테크·가전',
               '패션·잡화',
               '푸드', 
               '홈리빙',
               'Fri',
               'Mon',
               'Sat',
               'Sun',
               'Thu',
               'Tue',
               'Wed',
               '펀딩기간일_T',
               '일펀딩금액',
               '일글수',
              '문장당강조',
               '문장당밑줄',
              'Jan',
              'Feb',
              'Mar',
              'Apr',
              'May',
              'June',
              'July',
              'Aug',
              'Sep',
              'Oct',
              'Nov',
              'Dec',
              '시작1일차이_T',
              '시작5일차이_T',
              '시작10일차이_T',
              '시작15일차이_T',
              '시작20일차이_T',
              '시작25일차이_T',
              '마감1일차이_T',
              '마감5일차이_T',
              '마감10일차이_T',
              '마감15일차이_T',
              '마감20일차이_T',
              '마감25일차이_T']
data_f = data[feature_names]

In [ ]:
X = data_f
y = data.달성률_C

# 학습용/테스트용 데이터 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [ ]:
# 학습 모델 결정 및 학습 - RF Classification
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=300, 
                               max_depth = 5, n_jobs = -1)
rf_clf.fit(X_train, y_train)
rf_pred= rf_clf.predict(X_test)                     
                     
rf_clf.score(X_test, y_test)
                     
import sklearn.metrics as mt
print('accuracy', mt.accuracy_score(y_test,rf_pred) )
#print('precision', mt.precision_score(y_test,rf_pred) )
#print('recall', mt.recall_score(y_test,rf_pred) )
#print('f1', mt.f1_score(y_test,rf_pred) )
print(mt.classification_report(y_test,rf_pred))
print(mt.confusion_matrix(y_test,rf_pred))

In [ ]:
# 변수간 중요도 보기
feature=pd.DataFrame(wadizdata.columns)
importance=pd.DataFrame(rf_clf.feature_importances_)
중요도 =pd.concat([feature,importance],axis=1)
중요도